In [ ]:
# python 3.10.9에서 작성 되었습니다.

konlpy 사용시 참고 링크에 들어가서 설치하세요

참고 : https://wikidocs.net/22488

In [ ]:
# 패키지 불러오기

import pandas as pd
import re
import json
from konlpy.tag import Okt
from wordcloud import WordCloud
from tensorflow.keras.preprocessing.text import Tokenizer

okt = Okt()
tokenizer = Tokenizer()

In [ ]:
# 테스트
print('OKT 형태소 분석 :',okt.morphs("열심히 코딩한 당신, 연휴에는 여행을 가봐요"))
print('OKT 품사 태깅 :',okt.pos("열심히 코딩한 당신, 연휴에는 여행을 가봐요"))
print('OKT 명사 추출 :',okt.nouns("열심히 코딩한 당신, 연휴에는 여행을 가봐요")) 

In [ ]:
# 로우 데이터 불러오기 = df
df = pd.read_excel('dataframe.xlsx')
df.head(2)

In [ ]:
# 컬럼명 변경
df.columns = ['제목', '연수명', '분류', '만족도', '내용']
df.head(2)

In [ ]:
# label에 기본값으로 3
df['label'] = 3

In [ ]:
# 만족도 2이하면 0, 만족도 4이상이면 1
for i in range(len(df)):
    if df['만족도'][i]<=2:
        df['label'][i] = 0
    elif df['만족도'][i] >= 4:
        df['label'][i] = 1

In [ ]:
# label이 3이 아닌것만
df = df[df['label'] != 3]

In [ ]:
# index 초기화
df = df.reset_index(drop=True)
df

In [ ]:
# 만족도 5, 4 = df1
df1 = df[df['만족도'].isin([5, 4])]
df1

In [ ]:
# 만족도 2, 1 = df2
df2 = df[df['만족도'].isin([2, 1])]
df2

In [ ]:
# 내용만 추출
reviews = df['내용']
reviews

In [ ]:
# Mecab 작동 시험
from konlpy.tag import Mecab 
mecab_tokenizer = Mecab(dicpath=r"C:\mecab\share\mecab-ko-dic").morphs
print('mecab check :', mecab_tokenizer('어릴때보고 지금다시봐도 재밌어요ㅋㅋ'))

In [ ]:
# 결측치 확인 True가 있으면 nan값 있음
reviews.isna().unique()

In [ ]:
#결측치 제거
reviews = reviews.dropna()

In [ ]:
# Mecab으로 토큰화
from konlpy.tag import Mecab

tokens = [mecab_tokenizer(word) for word in reviews]

tokens = list(map(lambda x : " ".join(x), tokens))

tokens[:10]

In [ ]:
# 감성 사전 가져오기
with open('SentiWord_info.json', encoding='utf-8-sig', mode='r') as f: 
    SentiWord_info = json.load(f)

sentiword_dic = pd.DataFrame(SentiWord_info)

In [ ]:
# 감성사전으로 점수화

df3 = pd.DataFrame(columns=("review", "sentiment"))  # 리뷰별 극성을 저장하기 위한 데이터프레임 생성
idx = 0                                             # 다음 리뷰로 넘기기 위한 초기값
 
for token in tokens:                                # 전체 리뷰에서 문장 하나씩 가져옴 
    sentiment = 0                                   # 초기 감성값 0으로 설정
    print(idx)
    for i in range(0, len(sentiword_dic)):            # 감성사전의 모든 단어를 하나씩 선택
        if sentiword_dic.word[i] in token:              # 리뷰 문장에 감성 단어가 있는지 확인
            sentiment += int(sentiword_dic.polarity[i])   # 감성단어가 있다면 극성값 합계를 구함.
    df3.loc[idx] = [token, sentiment]                  # 리뷰별 극성값을 데이터프레임으로 쌓음
    idx += 1                                          # 다름 리뷰 문장으로 넘어감

In [ ]:
# 예측라벨 컬럼 생성
df3['pre_label'] = 0

In [ ]:
# sentiment가 음수,0면 0, 양수면 1
for i in range(len(df3)):
    if df3['sentiment'][i]<0:
        df3['pre_label'][i] = 0
    else:
        df3['pre_label'][i] = 1

In [ ]:
df3

In [ ]:
df3[df3['pre_label'] == 0]

In [ ]:
# 분류 평가
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, precision_score , recall_score, f1_score, roc_auc_score

print(confusion_matrix(df['label'], df3['pre_label']))
print("정확도:", accuracy_score(df['label'], df3['pre_label']))
print("정밀도:", precision_score(df['label'], df3['pre_label']))
print("재현율(민감도):", recall_score(df['label'], df3['pre_label']))
print("f1-score:", f1_score(df['label'], df3['pre_label']))
print("roc_auc_score:", roc_auc_score(df['label'], df3['pre_label']))

In [ ]:
# 파일로 내보내기
df3.to_excel('./df10.xlsx', index=False)

In [ ]:
#전처리 함수 만들기
def preprocessing(review, okt, remove_stopwords = False, stop_words =[]):
    #함수인자설명
    # review: 전처리할 텍스트
    # okt: okt객체를 반복적으로 생성하지 않고 미리 생성 후 인자로 받음\
    # remove_stopword: 불용어를 제거할지 여부 선택. 기본값 False
    # stop_words: 불용어 사전은 사용자가 직접 입력, 기본값 빈 리스트

    # 1. 한글 및 공백 제외한 문자 모두 제거
    review_text = re.sub('[^가-힣ㄱ-ㅎㅏ-ㅣ\\s]','',review)

    #2. okt 객체를 활용하여 형태소 단어로 나눔
    word_review = okt.morphs(review_text,stem=True)

    if remove_stopwords:
        #3. 불용어 제거
        word_review = [token for token in word_review if not token in stop_words]
    return word_review

In [ ]:
# 불용어 제거 = https://www.ranks.nl/stopwords/korean
f = open("Korean Stopwords.txt", "r", encoding='UTF8')

lst = []
while True:
    line = f.readline().strip()
    if not line: break
    lst.append(line)

In [ ]:
# 줄바꿈 제거 추가
lst2 = ['\n', '\n\n', '\n\n\n', '\n\n\n\n', '\n\n\n\n\n']
lst.extend(lst2)

In [ ]:
# 로우 데이터 = df 전처리
stop_words = lst
clean_review_df = []

for review in df['내용']:
    # 리뷰가 문자열인 경우만 전처리 진행
    if type(review) == str:
        clean_review_df.append(preprocessing(review,okt, remove_stopwords = True, stop_words = stop_words))
    else:
        clean_review_df.append([]) #str이 아닌 행은 빈칸으로 놔두기
clean_review_df

In [ ]:
# 만족도 5, 4 = df1 전처리
stop_words = lst
clean_review_df1 = []

for review in df1['내용']:
    # 리뷰가 문자열인 경우만 전처리 진행
    if type(review) == str:
        clean_review_df1.append(preprocessing(review,okt, remove_stopwords = True, stop_words = stop_words))
    else:
        clean_review_df1.append([]) #str이 아닌 행은 빈칸으로 놔두기
clean_review_df1

In [ ]:
# 만족도 2, 1 = df2 전처리
stop_words = lst
clean_review_df2 = []

for review in df2['내용']:
    # 리뷰가 문자열인 경우만 전처리 진행
    if type(review) == str:
        clean_review_df2.append(preprocessing(review,okt, remove_stopwords = True, stop_words = stop_words))
    else:
        clean_review_df2.append([]) #str이 아닌 행은 빈칸으로 놔두기
clean_review_df2

# 긍정/부정 단어 빈도 시각화

## 민수님 파일: 만족도로 분류한 긍정과 부정 그룹

In [ ]:
high_data=clean_review_df1
low_data=clean_review_df2

In [ ]:
#한 리스트로 결합
def list_smaller(l):
    oneline=[]
    for data in l:
        oneline+=data
    return oneline
high=list_smaller(high_data)
low=list_smaller(low_data)

In [ ]:
from collections import Counter

counter=Counter(high)
high_counter=pd.DataFrame.from_dict(data=dict(counter),orient='index')
high_counter.reset_index(inplace=True)
high_counter.columns = ['단어','갯수']

counter=Counter(low)
low_counter=pd.DataFrame.from_dict(data=dict(counter),orient='index')
low_counter.reset_index(inplace=True)
low_counter.columns = ['단어','갯수']

In [ ]:
# 갯수 많은 순서대로 확인
# high_counter.sort_values('갯수',ascending=False) 
# low_counter.sort_values('갯수',ascending=False)

In [ ]:
#불용어 사전(부정,긍정 그룹 중 공통 단어 삭제)
low_word=list(low_counter['단어'])
high_word=list(high_counter['단어'])

# 같은 문자 제거
def del_common_word(counter,word):
    handle=counter['단어'].apply(lambda x: True if x in word else False) #같은 단어면 True
    counter_comm=counter[handle]# 같은 문자가 어떤 것인지 마스킹 됨
    
    print(counter_comm[counter_comm['갯수']>5])# 출력-삭제 대상
    common=list(counter_comm[counter_comm['갯수']>5]['단어'])# high에서 없애야할 대상(조건: 5번 이상 반복 출현)
    return counter[counter['단어'].apply(lambda x: False if x in common else True)] #제거된 상황
high_counter_diff= del_common_word(high_counter,low_word)
low_counter_diff=del_common_word(low_counter,high_word)

In [ ]:
low_counter_diff.sort_values('갯수',ascending=False)

### 긍정적(점수 높음)

In [ ]:
high_counter_diff_dict=high_counter_diff.set_index('단어').to_dict()
high_review=high_counter_diff_dict['갯수']
high_review

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt

# 한글 폰트 설정(.ttf파일 다운로드 후 실행)
wordcloud_high = WordCloud('C:\Windows\Fonts'+'\malgun.ttf',max_words=50).generate_from_frequencies(high_review)
plt.imshow(wordcloud_high, interpolation='bilinear')
plt.axis('off')
plt.show()

## 부정적(점수 낮음)

In [ ]:
low_counter_diff_dict=low_counter_diff.set_index('단어').to_dict()
low_review=low_counter_diff_dict['갯수']
low_review

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt

# 한글 폰트 설정(.ttf파일 다운로드 후 실행)
wordcloud_low = WordCloud('C:\Windows\Fonts'+'\malgun.ttf',max_words=50).generate_from_frequencies(low_review)
plt.imshow(wordcloud_low, interpolation='bilinear')
plt.axis('off')
plt.show()

## 파이

In [ ]:
# 파이 출력을 위해 상위 10개만 파이로 출력
for_pie_high=high_counter_diff.sort_values('갯수',ascending=False)[:10]
for_pie_low=low_counter_diff.sort_values('갯수',ascending=False)[:10]

In [ ]:
from dash import Dash, html, dcc, callback, Output, Input
import plotly.express as px

app = Dash(__name__)

high_wordcloud=px.imshow(wordcloud_high)
low_wordcloud=px.imshow(wordcloud_low)
# high_pie=px.pie(for_pie_high,values='갯수',names='단어')
# low_pie=px.pie(for_pie_low,values='갯수',names='단어')
high_graph=px.bar(for_pie_high,y='갯수',x='단어')
low_graph=px.bar(for_pie_low,y='갯수',x='단어')


app.layout = html.Div([
    html.Div([
        html.H1(children='긍정_워드클라우드', style={'textAlign':'center'}),
        dcc.Graph(figure=high_wordcloud),
    ],style={'width': '600px', 'height':'600px','margin': '0px 50px 50px 50px','display':'inline-block'}),
    html.Div([
        html.H1(children='부정_워드클라우드', style={'textAlign':'center'}),
        dcc.Graph(figure=low_wordcloud),
    ],style={'width': '600px', 'height':'600px','margin': '0px 50px 50px 50px','display':'inline-block'}),
    html.Div([
        html.H1(children='긍정 상위 10개', style={'textAlign':'center'}),
        dcc.Graph(figure=high_graph),
    ],style={'width': '600px', 'height':'600px','margin': '0px 50px 50px 50px','display':'inline-block'}),
    html.Div([
        html.H1(children='부정 상위 10개', style={'textAlign':'center'}),
        dcc.Graph(figure=low_graph),
    ],style={'width': '600px', 'height':'600px','margin': '0px 50px 50px 50px','display':'inline-block'})
],style={'width': '2000px'})

if __name__ == '__main__':
    app.run(debug=True)

## 지수님 파일: 감성사전으로 분류한 긍정과 부정 그룹

In [ ]:
df3

In [ ]:
# 전처리
df3['sentiment'].unique() #문제 없음

In [ ]:
positive_con=df3['sentiment']>=0
positive=df3[positive_con] #긍정 그룹
negative=df3[-positive_con] #부정 그룹

In [ ]:
def unify(li):
    divided=[]
    for l in li:
        divided.append(l.split())
    return divided
positive_li=unify(list(positive['review']))
negative_li=unify(list(negative['review']))

In [ ]:
#한 리스트로 결합
def list_smaller(l):
    oneline=[]
    for data in l:
        oneline+=data
    return oneline
posi_l=list_smaller(positive_li)
nega_l=list_smaller(negative_li)

In [ ]:
#불용어 사전 
new_posi=[]
for l in posi_l:
    if l in lst: #korean stopword 삭제
        pass
    elif re.search('[^가-힣ㄱ-ㅎㅏ-ㅣ\\s]',l): # 한글이 아닌 내용 삭제
        pass
    else:
        new_posi.append(l) # 남은 내용만 모으기
new_posi

In [ ]:
new_nega=[]
for l in nega_l:
    if l in lst: #korean stopword 삭제
        pass
    elif re.search('[^가-힣ㄱ-ㅎㅏ-ㅣ\\s]',l): # 한글이 아닌 내용 삭제
        pass
    else:
        new_nega.append(l) # 남은 내용만 모으기
new_nega

In [ ]:
high=new_posi
low=new_nega

## 반복

In [ ]:
from collections import Counter

counter=Counter(high)
high_counter=pd.DataFrame.from_dict(data=dict(counter),orient='index')
high_counter.reset_index(inplace=True)
high_counter.columns = ['단어','갯수']

counter=Counter(low)
low_counter=pd.DataFrame.from_dict(data=dict(counter),orient='index')
low_counter.reset_index(inplace=True)
low_counter.columns = ['단어','갯수']

### 긍정적(감성사전분류)

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt

# 한글 폰트 설정(.ttf파일 다운로드 후 실행)
wordcloud_high = WordCloud('C:\Windows\Fonts'+'\malgun.ttf',max_words=50).generate_from_frequencies(high_review)
plt.imshow(wordcloud_high, interpolation='bilinear')
plt.axis('off')
plt.show()

## 부정적(감성사전 분류)

In [ ]:
low_counter_diff_dict=low_counter_diff.set_index('단어').to_dict()
low_review=low_counter_diff_dict['갯수']
low_review

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt

# 한글 폰트 설정(.ttf파일 다운로드 후 실행)
wordcloud_low = WordCloud('C:\Windows\Fonts'+'\malgun.ttf',max_words=50).generate_from_frequencies(low_review)
plt.imshow(wordcloud_low, interpolation='bilinear')
plt.axis('off')
plt.show()

In [ ]:
# 파이 출력을 위해 상위 10개만 파이로 출력
for_pie_high=high_counter_diff.sort_values('갯수',ascending=False)[:10]
for_pie_low=low_counter_diff.sort_values('갯수',ascending=False)[:10]

In [ ]:
from dash import Dash, html, dcc, callback, Output, Input
import plotly.express as px

app = Dash(__name__)

high_wordcloud=px.imshow(wordcloud_high)
low_wordcloud=px.imshow(wordcloud_low)
# high_pie=px.pie(for_pie_high,values='갯수',names='단어')
# low_pie=px.pie(for_pie_low,values='갯수',names='단어')
high_graph=px.bar(for_pie_high,y='갯수',x='단어')
low_graph=px.bar(for_pie_low,y='갯수',x='단어')


app.layout = html.Div([
    html.Div([
        html.H1(children='긍정_워드클라우드', style={'textAlign':'center'}),
        dcc.Graph(figure=high_wordcloud),
    ],style={'width': '600px', 'height':'600px','margin': '0px 50px 50px 50px','display':'inline-block'}),
    html.Div([
        html.H1(children='부정_워드클라우드', style={'textAlign':'center'}),
        dcc.Graph(figure=low_wordcloud),
    ],style={'width': '600px', 'height':'600px','margin': '0px 50px 50px 50px','display':'inline-block'}),
    html.Div([
        html.H1(children='긍정 상위 10개', style={'textAlign':'center'}),
        dcc.Graph(figure=high_graph),
    ],style={'width': '600px', 'height':'600px','margin': '0px 50px 50px 50px','display':'inline-block'}),
    html.Div([
        html.H1(children='부정 상위 10개', style={'textAlign':'center'}),
        dcc.Graph(figure=low_graph),
    ],style={'width': '600px', 'height':'600px','margin': '0px 50px 50px 50px','display':'inline-block'})
],style={'width': '2000px'})

if __name__ == '__main__':
    app.run(debug=True)

In [ ]:
high_counter_diff.sort_values('갯수',ascending=False) 

In [ ]:
low_counter_diff.sort_values('갯수',ascending=False) 